# NLP case study


The following table summarizes the datasets used throughout this notebook.

| dataset ID | dataset name| is_dialectical | is_MSA (Modern Standard Arabic) | is_balanced | num_of_tweets | num_of_pos_tweets | num_of_neg_tweets |
|-- | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | [arabic-sentiment-twitter-corpus](https://www.kaggle.com/mksaad/arabic-sentiment-twitter-corpus) | Yes | No/minority | Yes | 58,751 | 29,849 | 28,902  
| 2 |[SS2030](https://www.kaggle.com/snalyami3/arabic-sentiment-analysis-dataset-ss2030-dataset ) | Yes - Saudi dialect only | No/Minority | Yes | 4,252 | 2,436 | 1,816 
| 3 |[100k Arabic Reviews](https://www.kaggle.com/abedkhooli/arabic-100k-reviews ) | No/Minority | Yes | Yes | 66,666 | 33,333 | 33,333
| 4 | [ArSAS](https://homepages.inf.ed.ac.uk/wmagdy/resources.htm) | Yes - mixed dialects| No/Minority | Yes | 11,784 | 4,400 | 7,384

*(For a more detailed analysis of the datasets see [this](https://www.kaggle.com/yasmeenhany/dataset-analysis) companion notebook. )*



**[Importing the necessary modules](https://)**

In [138]:
import re
import io
# from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

%matplotlib inline

In [2]:
pip install camel-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 5.2 MB/s 
     |████████████████████████████████| 197 kB 33.3 MB/s 
     |████████████████████████████████| 418 kB 35.4 MB/s 
     |████████████████████████████████| 4.4 MB 38.3 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 596 kB 67.1 MB/s 
     |████████████████████████████████| 6.6 MB 23.7 MB/s 
  Created wheel for camel-kenlm: filename=camel_kenlm-2021.12.27-cp37-cp37m-linux_x86_64.whl size=2335128 sha256=7c9312f42866468c953a2f798dad0255f308d5a3634177046a9829764f7c51cd
  Stored in directory: /root/.cache/pip/wheels/db/72/74/982f8c435f15b7feaf6dc8a03e212ff34e93f1f2d747059332
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=7b90276fa5f02ef64e991a59126cc29f96afd9ec8cb852a21ccb42ef6abcf9fd
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe74

### Importing the dataset


In [139]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [140]:
pd.set_option('display.max_colwidth', 280)
train_neg = pd.read_csv("train_Arabic_tweets_negative_20190413.tsv", sep="\t", header=None,  quoting=csv.QUOTE_NONE)
train_pos = pd.read_csv("train_Arabic_tweets_positive_20190413.tsv", 
                        sep="\t", header=None,  quoting=csv.QUOTE_NONE)
train_neg.rename(columns={0:'label', 1:'tweet'}, inplace=True)
train_pos.rename(columns={0:'label', 1:'tweet'}, inplace=True)
train_neg['label'] = 0
train_pos['label'] = 1
train_df = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

### Visualizing the first 10 rows of the training dataset: 

In [141]:
train_df.head(10)

,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد 😭
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين احس فيه احد ناقصهم 💔 #Avlu
2,0,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال والاهلي تحت التاق 👇 #تحدي_اسرع_روقان وادخل في سحب قيمة ايفون X على…
3,0,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين على بكتيريا 🦠 فتنفجر 💥 و تموت . الأخيرة يبدو انها بكتيريا مقاومة فأخذ…
4,0,الدودو جايه تكمل علي 💔
5,0,أحببته حتى أقنعنى إن ما فات من العمر كان إنتظار له 🙈
6,0,يبدو ان دجلة اعتادت على التهام اجساد ابنائها من سبايكر للعبارة ..👆👆💔 ما سمعته .. ان البشر يعطش فيشرب الماء .. لم…
7,0,جالس أذاكر 😣
8,0,يكفي استخفاف بالعقل العربي هل حقا الأمن والمخابرات في ألمانيا غير قادرة على وقف اي عصابة أو الجريمة 🤔 وا…
9,0,ياليلل ترا اكلج كثير مشي 😭


### Feature engineering


In [142]:
from nltk.corpus import stopwords
import emoji
#Stats about Text
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words)/len(words))

def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

In [143]:
def add_features(df):
    new_df = df.copy()
    new_df['word_count'] = new_df['tweet'].apply(lambda x: len(str(x).split(" ")))
    new_df['char_count'] = new_df['tweet'].str.len() ## this also includes spaces
    new_df['avg_char_per_word'] = new_df['tweet'].apply(lambda x: avg_word(x))
    stop = stopwords.words('arabic')
    new_df['stopwords'] = new_df['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
    new_df['emoji_count'] = new_df['tweet'].apply(lambda x: emoji_counter(x))
    return new_df

In [27]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all-corpora


       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Downloading package cess_esp to /root/nltk_data...
       |   Package cess_esp is already up-to-date!
       | Downloading package chat80 to /root/nltk_data...
       |   Package chat80 is already up-to-date!
       | Downloading package city_database to /root/nltk_data...
       |   Package city_database is already u


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

### Adding new features to the train_df


In [144]:
train_df_enhanced = add_features(train_df)
train_df_enhanced

,label,tweet,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد 😭,12,56,3.750000,1,1
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين احس فيه احد ناقصهم 💔 #Avlu,15,72,3.866667,3,1
2,0,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال والاهلي تحت التاق 👇 #تحدي_اسرع_روقان وادخل في سحب قيمة ايفون X على…,18,112,5.277778,2,1
3,0,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين على بكتيريا 🦠 فتنفجر 💥 و تموت . الأخيرة يبدو انها بكتيريا مقاومة فأخذ…,21,115,4.523810,2,3
4,0,الدودو جايه تكمل علي 💔,5,22,3.600000,0,1
...,...,...,...,...,...,...,...
46995,1,السحب الليلة على الايفون .. رتويت للمرفقة وطبق الشروط 👇,10,55,4.600000,1,1
46996,1,😂 لابسة احمر ليه يا ست انتي ايه المناسبة 😂,10,42,3.300000,2,2
46997,1,كلاام جمييل تستاهل(من احبه الله جعل محبته ف قلوب البشر) 💙,11,57,4.272727,2,1
46998,1,- ألطف صورة ممكن تعبر عن رمضان 💙,8,32,3.125000,1,1


### Removing emojis

In [69]:
import string

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_emojis(text):
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)

def remove_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def remove_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [106]:
def clean_text(text):
    text = remove_punctuations(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = remove_emojis(text)
    text = normalize_arabic(text)
    text = remove_all_entities(text)
    text = remove_links(text)
    text.replace('_',' ')
    text = re.sub(r'[A-Za-z0-9]', '', text)
    return text

In [145]:
train_df_enhanced['tweet'] = train_df_enhanced['tweet'].apply(clean_text)

In [73]:
train_df

,label,tweet
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد 😭
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين احس فيه احد ناقصهم 💔 #Avlu
2,0,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال والاهلي تحت التاق 👇 #تحدي_اسرع_روقان وادخل في سحب قيمة ايفون X على…
3,0,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين على بكتيريا 🦠 فتنفجر 💥 و تموت . الأخيرة يبدو انها بكتيريا مقاومة فأخذ…
4,0,الدودو جايه تكمل علي 💔
...,...,...
46995,1,السحب الليلة على الايفون .. رتويت للمرفقة وطبق الشروط 👇
46996,1,😂 لابسة احمر ليه يا ست انتي ايه المناسبة 😂
46997,1,كلاام جمييل تستاهل(من احبه الله جعل محبته ف قلوب البشر) 💙
46998,1,- ألطف صورة ممكن تعبر عن رمضان 💙


In [146]:
train_df_enhanced


,label,tweet,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,0,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم بالزايد,12,56,3.750000,1,1
1,0,توقعت اذا جات داريا بشوفهم كاملين بس لي لحين احس فيه احد ناقصهم,15,72,3.866667,3,1
2,0,الاهليالهلال اكتب توقعك لنتيجه لقاء الهلال والاهلي تحت التاق تحدياسرعروقان وادخل في سحب قيمه ايفون علي,18,112,5.277778,2,1
3,0,نعمه المضادات الحيويه تضع قطرهمضاد بنسلين علي بكتيريا 🦠 فتنفجر و تموت الاخيره يبدو انها بكتيريا مقاومه فاخذ,21,115,4.523810,2,3
4,0,الدودو جايه تكمل علي,5,22,3.600000,0,1
...,...,...,...,...,...,...,...
46995,1,السحب اليله علي الايفون رتويت لمرفقه وطبق الشروط,10,55,4.600000,1,1
46996,1,لابسه احمر ليه يا ست انتي ايه المناسبه,10,42,3.300000,2,2
46997,1,كلام جميل تستاهلمن احبه اله جعل محبته ف قلوب البشر,11,57,4.272727,2,1
46998,1,الطف صوره مكن تعبر عن رمضان,8,32,3.125000,1,1


In [74]:
uploaded = files.upload()

Saving test_Arabic_tweets_negative_20190413.tsv to test_Arabic_tweets_negative_20190413.tsv
Saving test_Arabic_tweets_positive_20190413.tsv to test_Arabic_tweets_positive_20190413.tsv


### Importing the test set and applying modifications on it

In [147]:
test_pos = pd.read_csv("test_Arabic_tweets_positive_20190413.tsv", 
                       sep="\t", header=None,  quoting=csv.QUOTE_NONE)
test_neg = pd.read_csv("test_Arabic_tweets_negative_20190413.tsv", 
                       sep="\t", header=None,  quoting=csv.QUOTE_NONE)
test_pos.rename(columns={0:'label', 1:'tweet'}, inplace=True)
test_neg.rename(columns={0:'label', 1:'tweet'}, inplace=True)
test_neg['label']=0
test_pos['label']=1
test_df = pd.concat([test_neg, test_pos], axis=0).reset_index(drop=True)
test_df_enhanced = test_df.copy()
test_df_enhanced = add_features(test_df_enhanced)
test_df_enhanced['tweet'] = test_df_enhanced['tweet'].apply(clean_text)

In [148]:
test_df

,label,tweet
0,0,حتى الايتونز خربتوه مو صاحين انتو؟؟ 😭
1,0,واحد تبع النظام السوري يقول أن المخابرات السورية وراء تحطم مركبة إسرائيلية على سطح القمر 😳 أول مرة أعرف أن القمر أق…
2,0,الى متى التعامل السئ للخادمات وعدم احترامهم وكأنهم حشرات والله هالمواقف تصير بيننا ونشوفها المربية جات من بيتها مع…
3,0,رايح جاي ي طحلبي 🐸 #الهلال_الاهلي
4,0,تتمغط ومعها سداع 😫
...,...,...
11746,1,ربي اغفر لي و لوالدي و لأحبتي و للمؤمنين و المؤمنات و المسلمين و المسلمات الأحياء منهم و الأموات :)
11747,1,ربي يسعدنا وياكم 💛
11748,1,يتحدثون عن اخلاق حسين ونجوم فرقهم نهاياتهم الرياضية أليمة ومخجلة نختلف ونتفق حول حسين ولكن المؤكد أن صحيفته الأخلاق…
11749,1,صباحكم احتفالية لم تكتمل، وصاحب الاحتفاليه ماكمل المباراة برضوه..اجل بتغبن جمهور الهلال 😂


In [149]:
test_df_enhanced

,label,tweet,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,0,حتي الايتونز خربتوه مو صاحين انتو,7,37,4.428571,1,1
1,0,واحد تبع النظام السوري يقول ان المخابرات السوريه وراء تحطم مركبه اسراءيليه علي سطح القمر اول مره اعرف ان القمر اق,22,116,4.318182,5,1
2,0,الي متي التعامل السء لخادمات وعدم احترامهم وكانهم حشرات واله هالمواقف تصير بينا ونشوفها المربيه جات من بيتها مع,19,115,5.105263,2,0
3,0,رايح جاي ي طحلبي الهلالالاهلي,6,33,4.666667,1,1
4,0,تمغط ومعها سداع,4,18,3.750000,0,1
...,...,...,...,...,...,...,...
11746,1,ربي اغفر لي و لوالدي و لاحبتي و لمءمنين و المءمنات و المسلمين و المسلمات الاحياء منهم و الاموات,20,99,4.000000,8,0
11747,1,ربي يسعدنا وياكم,4,18,3.750000,0,1
11748,1,يتحدثون عن اخلاق حسين ونجوم فرقهم نهاياتهم الرياضيه اليمه ومخجله نختلف ونتفق حول حسين ولكن المءكد ان صحيفته الاخلاق,19,116,5.157895,3,0
11749,1,صباحكم احتفاليه لم تكتمل وصاحب الاحتفاليه ماكمل المباراه برضوهاجل بتغبن جمهور الهلال,13,89,5.923077,1,1


### Splitting the data

In [150]:
from sklearn.model_selection import train_test_split
#old train split
X = train_df.tweet.values
y = train_df.label.values
#processed train split
X_train_new = train_df_enhanced.tweet.values
y_train_new = train_df_enhanced.label.values

# The train val split is used by the DL approach but not classical ML
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1, random_state=1111)
X_train_clean , X_val_clean , y_train_clean , y_val_clean = train_test_split(X_train_new, y_train_new, 
                                                             test_size=0.1, random_state=1111)
#old test split
X_test = test_df.tweet.values
y_test = test_df.label.values
#processed test split
X_test_clean = test_df_enhanced.tweet.values
y_test_clean = test_df_enhanced.label.values

### Old model definition

In [151]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

def train_model(model, data, targets):
    text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
    ])
    text_clf.fit(data, targets)
    return text_clf
def get_accuracy(trained_model,X, y):
    predicted = trained_model.predict(X)
    accuracy = np.mean(predicted == y)
    return accuracy

### Old model metrics

In [83]:
old_model = train_model(MultinomialNB(), X_train, y_train)
test_accuracy       = get_accuracy(old_model,X_test, y_test)
validation_accuracy = get_accuracy(old_model,X_val, y_val)
training_accuracy   = get_accuracy(old_model,X_train, y_train)

print(f"test accuracy with MultinomialNB: {test_accuracy:.4f}")
print(f"validation accuracy with MultinomialNB: {validation_accuracy:.4f}")
print(f"Train accuracy with MultinomialNB: {training_accuracy:.4f}")

test accuracy with MultinomialNB: 0.7822
validation accuracy with MultinomialNB: 0.7828
Train accuracy with MultinomialNB: 0.8962


### New model definition

In [157]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV, KFold

def new_model(model, data, target):
    kf = KFold(n_splits=6, shuffle=True, random_state=1111)
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('model', model)    
    ])
    params = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__max_features': (None, 5000, 10000, 50000),
    'tfidf__ngram_range': ((1, 1), (1, 2), (1,3)),  
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2', None),
    'model__alpha': np.linspace(0.00001, 1, 20)
    }
    search = RandomizedSearchCV(estimator=pipe, param_distributions=params, cv=kf, 
                                n_iter=50, verbose=1, n_jobs = -1)
    search.fit(data, target)
    print(search.best_params_)
    return search

### Metrics from new model

In [158]:
better_model = new_model(MultinomialNB(), X_train_clean, y_train_clean)
test_accuracy       = get_accuracy(better_model,X_test_clean, y_test_clean)
validation_accuracy = get_accuracy(better_model,X_val_clean, y_val_clean)
training_accuracy   = get_accuracy(better_model,X_train_clean, y_train_clean)

print(f"test accuracy with MultinomialNB: {test_accuracy:.4f}")
print(f"validation accuracy with MultinomialNB: {validation_accuracy:.4f}")
print(f"Train accuracy with MultinomialNB: {training_accuracy:.4f}")

Fitting 6 folds for each of 50 candidates, totalling 300 fits
{'tfidf__use_idf': False, 'tfidf__norm': None, 'tfidf__ngram_range': (1, 3), 'tfidf__max_features': None, 'tfidf__max_df': 1.0, 'model__alpha': 0.26316526315789474}
test accuracy with MultinomialNB: 0.7984
validation accuracy with MultinomialNB: 0.7932
Train accuracy with MultinomialNB: 0.9797


In [88]:
better_model.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.5, ngram_range=(1, 3))),
                ('model', MultinomialNB(alpha=0.7894757894736841))])

In [89]:
uploaded = files.upload()

Saving Arabic Sentiment Analysis Dataset - SS2030.csv to Arabic Sentiment Analysis Dataset - SS2030.csv


### Uploading the SS2030 dataset

In [98]:
df_ss2030 = pd.read_csv("Arabic Sentiment Analysis Dataset - SS2030.csv")
# Rename columns to match convention
df_ss2030 = df_ss2030.rename(columns = {"text":"tweet", "Sentiment": "label"})

In [97]:
df_ss2030

,tweet,label
0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,1
1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. https://t.co/ps3qNw1CbB,1
2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (@Shubratanmyeh):\n \n ما زال التسجيل مستمر في دورة حقوق المرأة بعد الطلاق ✨ #وعيك_يحميك... https://t.co/c2NXzNCdLU,1
3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة العدل https://t.co/QUGzWwubFk,1
4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او ولي المراة من الاخطاء الشائعة \n \n هذا الكلام غلط في الشريعة والقانون\n فلا يوجد ولي للزوجة او المراة الا اذا كانت قاصرا ويكون الولي ابوها ...الخ وليس الزوج منهم\n نعم له حقوق عليها لكنها ليست ولاية\n الولاية مصطلح فقهي قانوني ...,1
...,...,...
4247,#غرد_بحبك_لمحمد_بن_سلمان ❤️,1
4248,#غرد_بحبك_لمحمد_بن_سلمان \n محمدبن سلمان احبه الله واختاره في هذا لوقت لشيءً هو يعلمه اما حبنا له فهو طبيعي ان يكون فوق العاده لاننا في حاجة لمثل هذا الرجل الملهم البطل المقدام حفظه الله وحفظ به امن وامان بلاد الحرمين الشريفين وامن الشعب السعودي الوفي,1
4249,#غرد_بحبك_لمحمد_بن_سلمان \n الله يحفظك يا ذخر الوطن ويخليك 💜,1
4250,#غرد_بحبك_لمحمد_بن_سلمان \n \n الله يحفظه ويحميه ويقويه وكلنا معه 💚,1


### Applying the modifications to the SS2030 dataset

In [107]:
df_ss2030_enhanced = df_ss2030.copy()
df_ss2030_enhanced['tweet'] = df_ss2030_enhanced['tweet'].apply(clean_text)
df_ss2030_enhanced = add_features(df_ss2030_enhanced)
df_ss2030_enhanced

,tweet,label,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,حقوق المراه,1,3,12,5.000000,0,0
1,حقوق المراه في الاسلام,1,7,25,4.750000,1,0
2,لجنه التنميه بشبرا ما زال التسجيل مستمر في دوره حقوق المراه بعد الطلاق وعيكيحميك,1,19,85,4.785714,3,0
3,حقوق المراه التي تضمنها لها وزاره العدل,1,10,42,4.714286,2,0
4,ولي امر الزوجه او ولي الزوجه او ولي المراه من الاخطاء الشاءعه هذا الكلام غلط في الشريعه والقانون فلا يوجد ولي لزوجه او المراه الا اذا كانت قاصرا ويكون الولي ابوها الخ وليس الزوج منهم نعم له حقوق عليها لكنها ليست ولايه الولايه مصطلح فقهي قانوني محد,1,50,250,4.276596,7,0
...,...,...,...,...,...,...,...
4247,غردبحبكلمحمدبنسلمان,1,1,19,19.000000,0,0
4248,غردبحبكلمحمدبنسلمان محمدبن سلمان احبه اله واختاره في هذا لوقت لشيء هو يعلمه اما حبنا له فهو طبيعي ان يكون فوق العاده لانا في حاجه لمثل هذا الرجل الملهم البطل المقدام حفظه اله وحفظ به امن وامان بلاد الحرمين الشريفين وامن الشعب السعودي الوفي,1,43,239,4.581395,8,0
4249,غردبحبكلمحمدبنسلمان اله يحفظك يا ذخر الوطن ويخليك,1,7,49,6.142857,1,0
4250,غردبحبكلمحمدبنسلمان اله يحفظه ويحميه ويقويه وكلنا معه,1,7,53,6.714286,0,0


In [159]:
from nltk.corpus.reader.rte import norm
pipe_with_params = Pipeline(steps=[
     ('tfidf',TfidfVectorizer(use_idf=False, norm=None, ngram_range=(1,3), max_df=1.0)),
     ('model', MultinomialNB(alpha=0.26316526315789474))                      
])
pipe_with_params.fit(X_train_clean, y_train_clean)
# {'tfidf__use_idf': False, 'tfidf__norm': None, 'tfidf__ngram_range': (1, 3), 'tfidf__max_features': None, 'tfidf__max_df': 1.0, 'model__alpha': 0.26316526315789474}

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(ngram_range=(1, 3), norm=None, use_idf=False)),
                ('model', MultinomialNB(alpha=0.26316526315789474))])

### Evaluating the new model vs the old model

In [160]:
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=10, shuffle=True, random_state=1111)
ss_2030_accuracy = get_accuracy(old_model,df_ss2030.tweet.values, df_ss2030.label.values)
print(f"df_ss2030 dataset accuracy with Multinomial NB: {ss_2030_accuracy:.4f}")
cv_score = cross_val_score(pipe_with_params,df_ss2030_enhanced.tweet.values, 
                           df_ss2030_enhanced.label.values, cv=kf)
print(f"df_ss2030_enhanced dataset accuracy with New model: {np.mean(cv_score):.4f}")

df_ss2030 dataset accuracy with Multinomial NB: 0.5910
df_ss2030_enhanced dataset accuracy with New model: 0.8862


### Uploading the arabic 100K review

In [112]:
uploaded = files.upload()

Saving ar_reviews_100k.tsv to ar_reviews_100k.tsv


In [114]:
df_reviews = pd.read_csv("ar_reviews_100k.tsv", delimiter="\t")
# Create a mapping for the labels such that we use the same convention across all datasets
label_mapping = {"Positive": 1, "Negative":0}
# Filter to only have pos and neg tweets, i.e: remove mixed tweets
df_reviews = df_reviews[df_reviews.label != "Mixed"]
df_reviews["label"] = df_reviews["label"].map(label_mapping)
# Rename columns to match convention
df_reviews = df_reviews.rename(columns = {"text":"tweet"})

In [115]:
df_reviews

,label,tweet
0,1,ممتاز نوعا ما . النظافة والموقع والتجهيز والشاطيء. المطعم
1,1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدولة يعشق ترابها. نحن نحب الإمارات. ومضات من فكر. نصائح لدولة تطمح بالصفوف الأولى و قائد لا يقبل إلا براحة شعبه وتوفر كل سب العيش الكريم. حكم و مواقف ونصائح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد. يستحق القراءة مرات كثيرة
2,1,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى هدوء جبال الشيشان .. للتعرف على حقيقة ما يجرى فى تلك البلاد من حروب ضاربة بحق المسلمين و جزء كبير من تاريخ تلك المنطقة. التضحية .. الرجولة .. الوفاء والكثير من القيم الأخرى اثبتت وجودها فى تلك الرواية البسيطة
3,1,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الازرق ميقراش احسن.. احمد مراد تخطى مرحلة ان القارئ يخلص الرواية وهو فاتح بؤه لمرحلة ان القارئ يخلص الرواية وهو محترم الكاتب.. اتقان مخيف.. بصرف النظر عن اخطاء لا تذكر ف الحوار.. انما احمد مراد سافر عاش حبة ف اوائل القرن العشرين وجه ي...
4,1,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل الخدمات مريح نفسيا. لا يوجد
...,...,...
99994,0,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من البداية..القصة تقليدية، الاحداث بطيئة ومملة والرواي أطول مما تستوجب.... وصلت لبعد منتصفها وقررت إنها متستحقش أضيع وقت تاني فيها
99995,0,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا يوجد خدمة افطار صباحي مستوي الفندق غير لائق
99996,0,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحاله أو مشهد بدون فكره للقصه
99997,0,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف عندة دقيق وزائد عن حد اللزوم.. هذا ثاني كتاب اقراءة للكتاب على أمل اني احب كتابته، لكن للأسف كان سيء زي الاول.


### Applying the feature engineering on the 100K dataset

In [116]:
df_reviews_clean = df_reviews.copy()
df_reviews_clean['tweet'] = df_reviews_clean['tweet'].apply(clean_text)
df_reviews_clean = add_features(df_reviews_clean)
df_reviews_clean

,label,tweet,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,1,متاز نوعا ما النظافه والموقع والتجهيز والشاطيء المطعم,8,53,5.750000,1,0
1,1,احد اسباب نجاح الامارات ان كل شخص في هذه الدوله يعشق ترابها نحن نحب الامارات ومضات من فكر نصاءح لدوله تطمح بالصفوف الاولي و قاءد لا يقبل الا براحه شعبه وتوفر كل سب العيش الكريم حكم و مواقف ونصاءح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد يستحق القراءه مرات كثيره,53,268,4.075472,11,0
2,1,هادفه وقويه تنقلك من صخب شوارع القاهره الي هدوء جبال الشيشان لتعرف علي حقيقه ما يجري في تلك البلاد من حروب ضاربه بحق المسلمين و جزء كبير من تاريخ تلك المنطقه التضحيه الرجوله الوفاء والكثير من القيم الاخري اثبت وجودها في تلك الروايه البسيطه,44,239,4.454545,11,0
3,1,خلصنا مبدءيا الي مستني ابهار زي الفيل الازرق ميقراش احسن احمد مراد تخطي مرحله ان القارء يخلص الروايه وهو فاتح بءه لمرحله ان القارء يخلص الروايه وهو محترم الكاتب اتقان مخيف بصرف النظر عن اخطاء لا تذكر ف الحوار انما احمد مراد سافر عاش حبه ف اواءل القرن العشرين وجه ياخدنا لهناك ...,106,544,4.141509,16,0
4,1,ياسات جلوريا جزء لا يتجزا من دبي فندق متكامل الخدمات مريح نفسيا لا يوجد,14,71,4.142857,3,0
...,...,...,...,...,...,...,...
99994,0,معرفش ليه كنت عاوزه اكملها وهي مش عاجباني من البدايهالقصه تقليديه الاحداث بطيءه ومله والرواي اطول ما تستوجب وصلت لبعد منتصفها وقرت انها متستحقش اضيع وقت تاني فيها,28,162,4.821429,3,0
99995,0,لا يستحق ان يكون في بوكنق لانه سيء لا شي لا يوجد خدمه افطار صباحي مستوي الفندق غير لاءق,19,87,3.631579,5,0
99996,0,كتاب ضعيف جدا ولم استمتع به في كل قصه سرد لحاله او مشهد بدون فكره لقصه,16,70,3.437500,3,0
99997,0,مله جدا محمد حسن علوان فنان بالكلمات والوصف عنده دقيق وزاءد عن حد الزوم هذا ثاني كتاب اقراءه لكتاب علي امل اني احب كتابته لكن لاسف كان سيء زي الاول,30,147,3.933333,5,0


In [117]:
df_reviews_clean

,label,tweet,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,1,متاز نوعا ما النظافه والموقع والتجهيز والشاطيء المطعم,8,53,5.750000,1,0
1,1,احد اسباب نجاح الامارات ان كل شخص في هذه الدوله يعشق ترابها نحن نحب الامارات ومضات من فكر نصاءح لدوله تطمح بالصفوف الاولي و قاءد لا يقبل الا براحه شعبه وتوفر كل سب العيش الكريم حكم و مواقف ونصاءح لكل فرد فينا ليس بمجرد كتاب سياسي كما كنت اعتقد يستحق القراءه مرات كثيره,53,268,4.075472,11,0
2,1,هادفه وقويه تنقلك من صخب شوارع القاهره الي هدوء جبال الشيشان لتعرف علي حقيقه ما يجري في تلك البلاد من حروب ضاربه بحق المسلمين و جزء كبير من تاريخ تلك المنطقه التضحيه الرجوله الوفاء والكثير من القيم الاخري اثبت وجودها في تلك الروايه البسيطه,44,239,4.454545,11,0
3,1,خلصنا مبدءيا الي مستني ابهار زي الفيل الازرق ميقراش احسن احمد مراد تخطي مرحله ان القارء يخلص الروايه وهو فاتح بءه لمرحله ان القارء يخلص الروايه وهو محترم الكاتب اتقان مخيف بصرف النظر عن اخطاء لا تذكر ف الحوار انما احمد مراد سافر عاش حبه ف اواءل القرن العشرين وجه ياخدنا لهناك ...,106,544,4.141509,16,0
4,1,ياسات جلوريا جزء لا يتجزا من دبي فندق متكامل الخدمات مريح نفسيا لا يوجد,14,71,4.142857,3,0
...,...,...,...,...,...,...,...
99994,0,معرفش ليه كنت عاوزه اكملها وهي مش عاجباني من البدايهالقصه تقليديه الاحداث بطيءه ومله والرواي اطول ما تستوجب وصلت لبعد منتصفها وقرت انها متستحقش اضيع وقت تاني فيها,28,162,4.821429,3,0
99995,0,لا يستحق ان يكون في بوكنق لانه سيء لا شي لا يوجد خدمه افطار صباحي مستوي الفندق غير لاءق,19,87,3.631579,5,0
99996,0,كتاب ضعيف جدا ولم استمتع به في كل قصه سرد لحاله او مشهد بدون فكره لقصه,16,70,3.437500,3,0
99997,0,مله جدا محمد حسن علوان فنان بالكلمات والوصف عنده دقيق وزاءد عن حد الزوم هذا ثاني كتاب اقراءه لكتاب علي امل اني احب كتابته لكن لاسف كان سيء زي الاول,30,147,3.933333,5,0


### Evaluating the accuracy of the two models

In [162]:
df_reviews_accuracy = get_accuracy(old_model,df_reviews.tweet.values, df_reviews.label.values)
print(f"df_reviews dataset accuracy with Multinomial NB: {df_reviews_accuracy:.4f}")
cv_scores_reviews = cross_val_score(pipe_with_params,df_reviews_clean.tweet.values, 
                            df_reviews_clean.label.values, cv=kf)
print(f"df_reviews_clean dataset accuracy with New model: {np.mean(cv_scores_reviews):.4f}")

df_reviews dataset accuracy with Multinomial NB: 0.6007
df_reviews_clean dataset accuracy with New model: 0.8462


### Uploading ArSAS 

In [120]:
uploaded = files.upload()

Saving ArSAS.txt to ArSAS.txt


In [121]:
df_arsas = pd.read_csv('ArSAS.txt', header = 0, delimiter = "\t")
# Filter to only have pos and neg tweets, i.e: remove mixed tweets
df_arsas_pos = df_arsas[df_arsas.Sentiment_label == 'Positive'] 
df_arsas_neg = df_arsas[df_arsas.Sentiment_label == 'Negative'] 
df_arsas = pd.concat([df_arsas_pos, df_arsas_neg], axis=0).reset_index(drop=True)
# Create a mapping for the labels such that we use the same convention across all datasets
label_mapping = {"Positive": int(1), "Negative":int(0)}
df_arsas["Sentiment_label"] = df_arsas["Sentiment_label"].map(label_mapping)
# Rename columns to match convention
df_arsas = df_arsas.rename(columns = {"Tweet_text":"tweet", "Sentiment_label":"label"})

In [122]:
df_arsas

,#Tweet_ID,tweet,Topic,label,Sentiment_label_confidence,Speech_act_label,Speech_act_label_confidence
0,929241870508724224,المباراة القـادمة #غانا x #مصر الجولة الأخيرة من المجموعة الـ 5 تصفيات كاس العالم 2018 روسـيا ترتيب مصر : المركز الاول 12 نقطة ( تم حسم التأهل للمونديال ) غــدا الساعة 5:30 ع قناة : بين ســبورت 1 تـــوقعاتكم لـ نتيجة الماتش .؟ 😀😁 https://t.co/RTQBNZXDqM,Event,1,0.38,Assertion,0.62
1,928615163250520065,وزير خارجية فرنسا عن منتدى شباب العالم: شعرت بارتياح وأنا أتابعه من باريس - https://t.co/hSvsbEaeUz #youm,Event,1,0.69,Assertion,1.00
2,929607749461250048,بسم الله نبدأ 👏 نغرد علي وسم 👇 👇 👇 👇 👇 ↩ #شباب_مصر_يريد ↪تحقيق أهداف ثورة يناير العيش والحرية والعدالة الاجتماعية والكرامة الإنسانية #تيم_المرابطون ✊ https://t.co/RDDVULxYuK,Long_Standing,1,1.00,Expression,1.00
3,928932760017866754,رحم الله شهداء ثورة يناير اللي بسببهم اتكسر الخوف جوانا من بلطجية الداخلية وبقو بالنسبالنا موظف عادي في الدولة,Long_Standing,1,0.35,Expression,1.00
4,929377209441284096,ليلة كروية أفريقية حاسمة في #تصفيات_كأس_العالم لأفريقيا، وكل التمنيات بالتوفيق لكل مم #المغرب و #تونس اللي يكفيهم التعادل للصعود. -- كوت ديفوار والمغرب .. 7:30 م تونس وليبيا .. 8:30 م -- https://t.co/KEZhIRxREP,Event,1,0.66,Assertion,0.67
...,...,...,...,...,...,...,...
11779,927250764514430977,على قولت مرتضى منصور باكابورت ضارب ... 😏,Entity,0,1.00,Expression,1.00
11780,927930867271524353,ماشى اوكى هترجع تيران وصنافير وهتفرج عن المعتقلين وبعدين؟!طيب والإرهاب والاسعار والتعليم والصحة والسياحة والاستثمار والاقتصاد مش هما الاولى,Long_Standing,0,0.66,Expression,1.00
11781,929273539781328896,انت ودولتكم سرطان هذه الامه نسأل العظيم ان يرد كيدكم في نحوركم ويخرج ذنب الدماء اللي قتلت باسبب تدخلتكم في زعزعة واستقرار شعوب الربيع العربي وقتل الالف في رابعه واليمن وليبيا الله جبار منتقم وتلك الايام ندولها بين الناس,Long_Standing,0,1.00,Expression,1.00
11782,927902747223044097,أزمة المثقف العربي.. في ظل إشراقات”الربيع العربي” https://t.co/561tHwc3CO https://t.co/urHQlHljsN,Long_Standing,0,0.69,Assertion,0.69


In [129]:
df_arsas_clean = df_arsas.copy()
df_arsas_clean = df_arsas_clean.drop(columns=['#Tweet_ID','Topic','Sentiment_label_confidence','Speech_act_label','Speech_act_label_confidence'], axis=1)
df_arsas_clean['tweet'] = df_arsas_clean['tweet'].apply(clean_text)
df_arsas_clean = add_features(df_arsas_clean)
df_arsas_clean

,tweet,label,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,المباراه القادمه غانا مصر الجوله الاخيره من المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التاهل لمونديال غدا الساعه ع قناه بين سبورت توقعاتكم ل نتيجه الماتش,1,39,189,4.718750,5,0
1,وزير خارجيه فرنسا عن منتدي شباب العالم شعرت بارتياح وانا اتابعه من باريس,1,15,74,4.615385,2,0
2,بسم اله نبدا نغرد علي وسم ↩ شبابمصريريد ↪تحقيق اهداف ثوره يناير العيش والحريه والعداله الاجتماعيه والكرامه الانسانيه تيمالمرابطون,1,20,130,5.842105,1,2
3,رحم اله شهداء ثوره يناير الي بسبهم اتكسر الخوف جوانا من بلطجيه الداخليه وبقو بالنسبالنا موظف عادي في الدوله,1,19,107,4.684211,3,0
4,ليله كرويه افريقيه حاسمه في تصفياتكاسالعالم لافريقيا وكل التمنيات بالتوفيق لكل م المغرب و تونس الي يكفيهم التعادل لصعود كوت ديفوار والمغرب م تونس وليبيا م,1,29,157,4.961538,5,0
...,...,...,...,...,...,...,...
11779,علي قولت مرتضي منصور باكابورت ضارب,0,6,34,4.833333,0,0
11780,ماشي اوكي هترجع تيران وصنافير وهتفرج عن المعتقلين وبعدينطيب والارهاب والاسعار والتعليم والصحه والسياحه والاستثمار والاقتصاد مش هما الاولي,0,19,137,6.263158,2,0
11781,انت ودولتكم سرطان هذه الامه نسال العظيم ان يرد كيدكم في نحوركم ويخرج ذنب الدماء الي قتلت باسب تدخلتكم في زعزعه واستقرار شعوب الربيع العربي وقتل الالف في رابعه واليمن وليبيا اله جبار منتقم وتلك الايام ندولها بين الناس,0,39,216,4.564103,5,0
11782,ازمه المثقف العربي في ظل اشراقاتالربيع العربي,0,9,47,5.571429,1,0


In [130]:
df_arsas_clean

,tweet,label,word_count,char_count,avg_char_per_word,stopwords,emoji_count
0,المباراه القادمه غانا مصر الجوله الاخيره من المجموعه ال تصفيات كاس العالم روسيا ترتيب مصر المركز الاول نقطه تم حسم التاهل لمونديال غدا الساعه ع قناه بين سبورت توقعاتكم ل نتيجه الماتش,1,39,189,4.718750,5,0
1,وزير خارجيه فرنسا عن منتدي شباب العالم شعرت بارتياح وانا اتابعه من باريس,1,15,74,4.615385,2,0
2,بسم اله نبدا نغرد علي وسم ↩ شبابمصريريد ↪تحقيق اهداف ثوره يناير العيش والحريه والعداله الاجتماعيه والكرامه الانسانيه تيمالمرابطون,1,20,130,5.842105,1,2
3,رحم اله شهداء ثوره يناير الي بسبهم اتكسر الخوف جوانا من بلطجيه الداخليه وبقو بالنسبالنا موظف عادي في الدوله,1,19,107,4.684211,3,0
4,ليله كرويه افريقيه حاسمه في تصفياتكاسالعالم لافريقيا وكل التمنيات بالتوفيق لكل م المغرب و تونس الي يكفيهم التعادل لصعود كوت ديفوار والمغرب م تونس وليبيا م,1,29,157,4.961538,5,0
...,...,...,...,...,...,...,...
11779,علي قولت مرتضي منصور باكابورت ضارب,0,6,34,4.833333,0,0
11780,ماشي اوكي هترجع تيران وصنافير وهتفرج عن المعتقلين وبعدينطيب والارهاب والاسعار والتعليم والصحه والسياحه والاستثمار والاقتصاد مش هما الاولي,0,19,137,6.263158,2,0
11781,انت ودولتكم سرطان هذه الامه نسال العظيم ان يرد كيدكم في نحوركم ويخرج ذنب الدماء الي قتلت باسب تدخلتكم في زعزعه واستقرار شعوب الربيع العربي وقتل الالف في رابعه واليمن وليبيا اله جبار منتقم وتلك الايام ندولها بين الناس,0,39,216,4.564103,5,0
11782,ازمه المثقف العربي في ظل اشراقاتالربيع العربي,0,9,47,5.571429,1,0


### Evaluating the models on the df_arsas :

In [161]:
df_arsas_accuracy = get_accuracy(old_model,df_arsas.tweet.values, df_arsas.label.values)
print(f"df_arsas dataset accuracy with Multinomial NB: {df_arsas_accuracy:.4f}")
cv_scores_arsas = cross_val_score(pipe_with_params,df_arsas_clean.tweet.values, 
                            df_arsas_clean.label.values, cv=kf)
print(f"df_arsas_clean dataset accuracy with New model: {np.mean(cv_scores_arsas):.4f}")

df_arsas dataset accuracy with Multinomial NB: 0.6530
df_arsas_clean dataset accuracy with New model: 0.8610


# Conclusion


### Summary of accuracy results using the old model and the enhanced one:


| Dataset | Old model | New model | 
| :---: | :---: |:---: |
| arabic-sentiment-twitter-corpus test subset | 0.7822 | 0.7982 | 
| SS2030 |  0.5910 | **0.8862**
| 100k reviews | 0.6007 | **0.8462**
| ArSAS | 0.6530 | **0.8610** 
    

    

